# porto-seguro-safe-driver-prediction

# import

In [ ]:
import numpy as np
import pandas as pd
import tqdm
import os
from collections import deque
from collections import Counter
import codecs
import copy
from sklearn.metrics import confusion_matrix
import sys
from platform import python_version
from datetime import datetime
import time
import shutil
from sklearn.preprocessing import OneHotEncoder
import csv
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE

from catboost import Pool,CatBoostClassifier

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier


import matplotlib
from matplotlib import pylab as plt
import seaborn as sns


# environment

In [ ]:
print(python_version())

In [ ]:
%precision %08.2f

In [ ]:
sys.getdefaultencoding()

In [ ]:
%autosave 0

In [ ]:
#最大表示列数の指定（ここでは2000列を指定）
pd.set_option('display.max_columns', 2000)

In [ ]:
#最大表示行数の指定（ここでは200行を指定）
pd.set_option('display.max_rows', 200)

# define

In [ ]:
# n_estimators:ここを増やすと精度は上がるが遅い。。。
ESTIMATOR_NUM=10
RANDOM_STATE_VAL = 0
target_col_name = "target"
id_col_name = "id"
CV_VAL = 5

TEST_SIZE_VAL=0.2
#FEATURE_NUM = 950
#FEATURE_NUM = 150
FEATURE_NUM = 200

SELECTOR_STEP = .05


PROBA_THRESHOLD=0.7
#PROBA_THRESHOLD=0.6
#PROBA_THRESHOLD=0.5

# カテゴリ変数に変換する対象のカラム名
CONVERT_TO_CATEGORY_COL_NAME = ["ps_reg_01", "ps_reg_02", "ps_reg_03", "ps_car_12", "ps_car_13" ,"ps_car_14", "ps_car_15"]


# 特徴量-targetの相関の絶対値が小さいので消すしきい値
#DEL_FEATURE_TARGET_CORR_COL_THRESHOLD = 0.001
DEL_FEATURE_TARGET_CORR_COL_THRESHOLD = 0.005
#DEL_FEATURE_TARGET_CORR_COL_THRESHOLD = 0.01


# 特徴量同士の相関の絶対値が大きいので消すしきい値
DEL_BIG_CORR_COL_THRESHOLD = 0.8

# # 歪度の絶対値が大きいので消すカラム数
# DEL_SKEW_COL_NUM = 3
# 歪度の絶対値が大きいので消す対象のカラムのしきい値
DEL_SKEW_THRESHOLD = 20.0

# 尖度の絶対値が大きいので消す対象のカラムのしきい値
#DEL_KURT_THRESHOLD = 100.0
#DEL_KURT_THRESHOLD = 150.0
DEL_KURT_THRESHOLD = 500.0


# 不正値をnp.NaNに置換するカラム名のリスト
#ILLEGAL_REPLACE_NAN_COL_NAME_LIST = ["ps_ind_05_cat", "ps_car_01_cat", "ps_reg_03", "ps_car_12", "ps_car_14"]
ILLEGAL_REPLACE_NAN_COL_NAME_LIST = ["ps_ind_02_cat", "ps_ind_04_cat", "ps_ind_05_cat",
                                     "ps_reg_03",
                                     "ps_car_01_cat", "ps_car_02_cat", "ps_car_03_cat", "ps_car_05_cat", "ps_car_07_cat", "ps_car_09_cat", 
                                     "ps_car_11", "ps_car_12", "ps_car_14"]


# 外れ値をnp.NaNに置換するカラム名のリスト
#OUTLIER_REPLACE_NAN_COL_NAME_LIST = ["ps_reg_03", "ps_car_12", "ps_car_14"]
#OUTLIER_REPLACE_NAN_COL_NAME_LIST = ["ps_reg_03", "ps_car_13"]
OUTLIER_REPLACE_NAN_COL_NAME_LIST = ["ps_car_13"]



# function

In [ ]:
def dispalyDf(readDf, dispFlag, row_num, dispLabel):
    print("========================= {} =========================".format(dispLabel))
    row_num , col_num = np.shape(readDf)
    print("row_num = {}, col_num = {}".format(row_num, col_num))
    if dispFlag:
        display(readDf.head(row_num))
    print("++++++ None Count All++++++")
    print(readDf.isnull().values.sum())
    print("++++++ None Count ++++++")
    print(readDf.isnull().sum())
    print("++++++ Col Type ++++++")
    print(readDf.dtypes)
    print("++++++ Unique By Col（Noneは含まないので注意） ++++++")
    for col_name in readDf.columns.tolist():
        print("{} : {}".format(col_name, readDf[col_name].nunique()))

In [ ]:
################################################
######## チェック関数
################################################
def chkVal(expect_label, expect_num, act_label, act_num):
    if expect_num != act_num:
        raise Exception("Num Error !! expect({}): {}, act({}) : {}"
                        .format(expect_label, expect_num, act_label, act_num))


In [ ]:
def getStandardScalerDf(inputDf):
    row_num, col_num = np.shape(inputDf)
    
    # スケール変換
    scaler = StandardScaler()
    
    val_array = scaler.fit_transform(np.array(inputDf.values.tolist()))
    standardScDf = pd.DataFrame(val_array, dtype=np.float, columns = inputDf.columns.tolist())
    
    sc_row_num, sc_col_num = np.shape(standardScDf)

    # 行数チェック
    chkVal("expect_row_num", row_num, "sc_row_num", sc_row_num)
    # 列数チェック
    chkVal("expect_col_num", col_num, "sc_col_num", sc_col_num)

    
    del val_array
    return standardScDf


In [ ]:
# test用のカラムのDataFrame
def getAdjustTestColDf(inputDf, train_col_list):
    not_enough_col_list = set(train_col_list) - set(inputDf.columns.tolist())
    print("# trainに有ってtestにないカラム名のリスト")
    print("{} : {}".format(len(not_enough_col_list), not_enough_col_list))
        
    dataLen = len(inputDf)
        
    retDf = inputDf.join(
        pd.DataFrame({col_name : [0]*dataLen  for col_name  in not_enough_col_list },
                     dtype=np.float, columns=not_enough_col_list)
    ).loc[:, train_col_list]
    
    # カラムリストが一致するかチェック
    if retDf.columns.tolist() != train_col_list:
        raise Exception("col_name_list not Match train and test.")

    # データ行数が一致するかチェック
    chkVal("dataLen", dataLen, "retDf", len(retDf))

    return retDf

# process

In [ ]:
np.random.seed(RANDOM_STATE_VAL)

In [ ]:
act_train_read_Df = pd.read_csv('input/train.csv',dtype={id_col_name:object})
act_test_read_Df = pd.read_csv('input/test.csv',dtype={id_col_name:object})

In [ ]:
act_train_Df = act_train_read_Df.copy()
act_test_Df = act_test_read_Df.copy()

In [ ]:
dispalyDf(act_train_Df, True, 10, "act_train_Df")

In [ ]:
dispalyDf(act_test_Df, True, 10, "act_test_Df")

In [ ]:
#################################################

In [ ]:
# train:データのDataFrameと、ターゲットのデータフレームに分ける。
train_target_Ser = act_train_Df[target_col_name]
act_train_Df = act_train_Df.loc[:,[col_name for col_name in act_train_Df.columns.tolist() if col_name not in [id_col_name,target_col_name]]]
print(set(train_target_Ser))

In [ ]:
train_target_Ser.value_counts()

In [ ]:
# test:idとデータのDataFrameを抽出。
test_id_Df = act_test_Df.loc[:,[id_col_name]]
act_test_Df = act_test_Df.loc[:,[col_name for col_name in act_test_Df.columns.tolist() if col_name != id_col_name]]

In [ ]:
print("trainのカラム数 = ",len(act_train_Df.columns.tolist()))
print("testのカラム数 = ",len(act_test_Df.columns.tolist()))

In [ ]:
#############################################################

In [ ]:
# ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼

In [ ]:
kurtDf = act_train_Df.kurt().reset_index()
kurtDf.columns=["col_name","kurt"]
#skews_col_list = skewDf[skewDf["skew"]>DEL_SKEW_THRESHOLD]["col_name"].values.tolist()

In [ ]:
kurtDf[kurtDf["kurt"].abs()>10]

In [ ]:
minSer = act_train_Df.loc[:,].min()
print(minSer[minSer==-1])

In [ ]:
act_train_Df.corr()

In [ ]:
tempDf = act_train_Df.corr().abs().min().reset_index()
tempDf.columns=["index","val"]
tempDf.head()

In [ ]:
tempDf.sort_values("val").tail(25)

In [ ]:
act_train_Df.corr().abs().min()[-30:]

In [ ]:
threshold = 0.5

feat_corr = set()
corr_matrix = act_train_Df.corr()
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > threshold:
            feat_name = corr_matrix.columns[i]
            feat_corr.add(feat_name)

print(len(set(feat_corr)))
# -> 658

In [ ]:
list(set(feat_corr))

In [ ]:
act_train_Df["ps_reg_01"]

In [ ]:
tempDf = act_train_Df.corr().abs().min().reset_index()




In [ ]:
tempDf.columns=["col_name","corr"]
tempDf.head()

In [ ]:
act_train_Df["ps_ind_12_bin"].unique()

In [ ]:
act_train_Df["ps_ind_12_bin"].value_counts()

In [ ]:
act_train_Df["ps_ind_14"].unique()

In [ ]:
act_train_Df["ps_ind_14"].value_counts()

In [ ]:
act_train_Df["ps_car_10_cat"].unique()

In [ ]:
act_train_Df["ps_car_10_cat"].value_counts()

In [ ]:
act_train_Df["ps_car_02_cat"].value_counts()

In [ ]:
col_name_list = []
for col_name in act_train_Df.columns.tolist():
    if  "_cat"  in col_name or "_bin"  in col_name:
        if len([val for val in act_train_Df[col_name].values.tolist() if val <0]) > 0:
            print("========== {} ==========".format(col_name))
            print(act_train_Df[col_name].value_counts())
            col_name_list.append(col_name)

            
print(col_name_list)

In [ ]:
# グラフのサイズを指定
plt.figure(figsize=(20, 4))

# 箱ひげ図を表示
plt.subplot(1, 4, 1)
sns.boxplot(data=act_train_Df["ps_reg_03"])

plt.subplot(1, 4, 2)
sns.boxplot(data=act_train_Df["ps_car_12"])

plt.subplot(1, 4, 3)
sns.boxplot(data=act_train_Df["ps_car_13"])

plt.subplot(1, 4, 4)
sns.boxplot(data=act_train_Df["ps_car_14"])


In [ ]:
# グラフのサイズを指定
plt.figure(figsize=(20, 4))

# ヒストグラムを表示
plt.subplot(1, 4, 1)
plt.hist(act_train_Df["ps_reg_03"], bins=4)

plt.subplot(1, 4, 2)
plt.hist(act_train_Df["ps_car_12"], bins=20)

plt.subplot(1, 4, 3)
plt.hist(act_train_Df["ps_car_13"], bins=20)

plt.subplot(1, 4, 4)
plt.hist(act_train_Df["ps_car_14"], bins=20)


In [ ]:
act_train_Df_cp = act_train_Df.copy()

In [ ]:
tmpSer=act_train_Df_cp["ps_ind_05_cat"].replace(-1, np.nan)

In [ ]:
unique_list = act_train_Df_cp[tmpSer.notnull()]["ps_ind_05_cat"].unique().tolist()

In [ ]:
unique_list

In [ ]:
totalDf = None
for col_name in act_train_Df.columns.tolist():
    tempDf = act_train_Df.loc[:,[col_name]].join(pd.DataFrame(train_target_Ser, dtype=int)).corr().abs().loc[[col_name],[train_target_Ser.name]]
    if totalDf is None:
        totalDf = tempDf.copy()
    else:
        totalDf = pd.concat([totalDf, tempDf])
    del tempDf


In [ ]:
totalDf = totalDf.sort_values("target")

In [ ]:
totalDf.head(200)

In [ ]:
totalDf[totalDf[train_target_Ser.name]<DEL_FEATURE_TARGET_CORR_COL_THRESHOLD].index.tolist()

In [ ]:
# ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲

In [ ]:
#############################################################

In [ ]:
def percentile_Arr(val_Arr):
    val_Arr = val_Arr*100
    p25_val = np.percentile(val_Arr,25)
    p50_val = np.percentile(val_Arr,50)
    p75_val = np.percentile(val_Arr,75)

    # 低い順に評価していく。
    # 25%未満
    val_Arr = np.where(val_Arr< p25_val,1,val_Arr)
    #25%以上50%未満
    val_Arr = np.where(((val_Arr>= p25_val) & ((val_Arr<p50_val))) ,2,val_Arr)
    #50%以上75%未満
    val_Arr = np.where(((val_Arr>= p50_val) & ((val_Arr<p75_val))) ,3,val_Arr)
    #75%以上
    val_Arr = np.where(val_Arr>= p75_val ,4,val_Arr)
    
    return val_Arr

In [ ]:
def get_del_colmns(input_Df, target_Ser):
    drop_columns = []
    
    # 値のユニーク数が１のカラムをdropリストに追加する
    drop_columns.extend([col_name for col_name in input_Df.columns.tolist()
                        if input_Df[col_name].nunique() == 1 not in drop_columns])

#     # 値が同じカラムをdropリストに追加する。
#     drop_columns = getDropListEqualVal(input_Df, drop_columns)

    # 相関が大きい特徴量の片方を消す
    feat_corr = set()
    corr_matrix = input_Df.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > DEL_BIG_CORR_COL_THRESHOLD:
                feat_name = corr_matrix.columns[i]
                feat_corr.add(feat_name)
    
    corr_col_list = list(set(feat_corr))
    print("del corr_col_list = ",corr_col_list)
    drop_columns.extend(corr_col_list)
    
    # targetとの相関が小さい特徴量を消す
    target_col_name = target_Ser.name
    totalDf = None
    for col_name in input_Df.columns.tolist():
        tempDf = input_Df.loc[:,[col_name]].join(pd.DataFrame(target_Ser, dtype=int)).corr().abs().loc[
            [col_name],[target_col_name]]
        if totalDf is None:
            totalDf = tempDf.copy()
        else:
            totalDf = pd.concat([totalDf, tempDf])
        del tempDf
    to_target_smal_corr_drop_col_list = totalDf[totalDf[target_col_name] < DEL_FEATURE_TARGET_CORR_COL_THRESHOLD].index.tolist()
    print("del to_target_smal_corr_drop_col_list = ",to_target_smal_corr_drop_col_list)
    drop_columns.extend(to_target_smal_corr_drop_col_list)
    del totalDf
    
    # 歪度の絶対値が大きいカラムをdropリストに追加する。
    skewDf = input_Df.skew().reset_index()
    skewDf.columns=["col_name","skew"]
    skews_col_list = skewDf[skewDf["skew"].abs()>DEL_SKEW_THRESHOLD]["col_name"].values.tolist()
    del skewDf
    print("del skews_col_list = ",skews_col_list)
    drop_columns.extend(skews_col_list)

    # 尖度の絶対値が大きいカラムをdropリストに追加する。
    kurtDf = input_Df.kurt().reset_index()
    kurtDf.columns=["col_name","kurt"]
    kurts_col_list = kurtDf[kurtDf["kurt"].abs()>DEL_KURT_THRESHOLD]["col_name"].values.tolist()
    del kurtDf
    print("del kurts_col_list = ",kurts_col_list)
    drop_columns.extend(kurts_col_list)

    
    # カラム名の重複を無くす。
    drop_columns = list(set(drop_columns))
    print("drop_columns = ",drop_columns)

    return drop_columns
            

In [ ]:
def get_dropna_Df(inputDf, target_Ser):
    
    input_colums = inputDf.columns.tolist()
    
    retDf = inputDf.copy()
    targetDf = pd.DataFrame(target_Ser, dtype=int)
    retDf = retDf.join(targetDf)

    for col_name in ILLEGAL_REPLACE_NAN_COL_NAME_LIST:
        print("================== {} =================".format(col_name))
        print(retDf[col_name].value_counts())
        tmpSer=retDf[col_name].replace(-1, np.nan)
        retDf[col_name] = tmpSer

    # targetが1
    # または
    # nanを含まない行かつtargetが0
    retDf = retDf[(retDf[target_Ser.name] == 1) | (retDf.notnull().all(axis=1)  & (retDf[target_Ser.name] == 0))]

    # nanを-1に戻す
    retDf=retDf.replace(np.nan, -1)

    ##
    del inputDf
    
    return retDf.loc[:,input_colums], retDf[target_Ser.name]

In [ ]:
def get_fillna_Df(inputDf):
    
    retDf = inputDf.copy()

    
#     for col_name in ['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat',
#                      'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_05_cat', 'ps_car_07_cat', 'ps_car_09_cat']:
#     for col_name in ['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat',
#                      'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_07_cat', 'ps_car_09_cat']:
#    for col_name in ['ps_ind_05_cat', 'ps_car_01_cat']:
    for col_name in ILLEGAL_REPLACE_NAN_COL_NAME_LIST:
        print("================== {} =================".format(col_name))
        print(retDf[col_name].value_counts())
        tmpSer=retDf[col_name].replace(-1, np.nan)

        if  col_name.endswith("_cat"):
            # NaNをランダムチョイスで埋める。
            unique_list = retDf[tmpSer.notnull()][col_name].unique().tolist()
            tmpSer=np.where(tmpSer.isnull(), np.random.choice(unique_list, 1)[0], tmpSer)
        else:
            # NaNを前後の値で埋める。
#             tmpSer = tmpSer.fillna(method="ffill")
#             tmpSer = tmpSer.fillna(method="bfill")
#             # 平均値で埋める
#              tmpSer = tmpSer.fillna(tmpSer.mean())
            # 中央値で埋める
            tmpSer = tmpSer.fillna(tmpSer.median())


        retDf[col_name] = tmpSer
        print(retDf[col_name].value_counts())
    ##
    del inputDf
    
    return retDf

In [ ]:
###### 前処理 ######
def getBeroreProcessDf(input_Df, drop_columns, trainFlag, train_val_range_dict, target_Ser):
    retDf = input_Df.copy()

    #### 訓練時のみの処理
    if trainFlag:
        # 欠損値の行を削除する。
        retDf, target_Ser = get_dropna_Df(retDf, target_Ser)
    ####
    
    # 欠損値を補完する。
    retDf = get_fillna_Df(retDf)
    
#     # 外れ値をnanに変換して補完する。
#     retDf = get_outler_replace_Df(retDf)
    

#     # 連続変数をカテゴリ変数に変換する
#     for col_name in CONVERT_TO_CATEGORY_COL_NAME:
#         retDf[col_name] = percentile_Arr(retDf[col_name].values).astype(np.int32)
#         retDf = retDf.rename(columns={col_name: '{}_cat'.format(col_name)})

    #### 訓練時のみの処理
    if trainFlag:
        # 削除対象カラム名の取得
        drop_columns = get_del_colmns(input_Df, target_Ser)
    
    ####
    
    retDf = retDf.drop(drop_columns, axis=1)
    
    columns = retDf.columns.tolist()


        
###########        
#     for col_name in columns:
#         if retDf[col_name].dtype == object:
#             # 文字列のカラムの加工処理
#             retDf[col_name] = retDf[col_name].fillna('type 0')
#             retDf[col_name] = retDf[col_name].apply(lambda x: x.split(' ')[1])
            
#         # int64に変換する。
#         retDf[col_name] = pd.to_numeric(retDf[col_name])
#         elif retDf[col_name].dtype == np.int64:
#             # float64に変換する。
#             retDf[col_name] = retDf[col_name].astype(np.float64)

            
#         if trainFlag:
#             # train時
#             # 最小と最大を取得
#             train_val_range_dict[col_name] = [
#                 retDf[col_name].min(), retDf[col_name].max()
#             ]
#         else:
#             # test時
#             # 最小〜最大の範囲内に収める
#             retDf[col_name]  = getSerTestValInRange(retDf[col_name] , train_val_range_dict[col_name])
        
     
#     # 桁数が大きい値をlogにする
#     retDf["var38"] = np.log(retDf["var38"])
    
    return retDf, drop_columns, train_val_range_dict, target_Ser
    

In [ ]:
act_train_Df_2, drop_columns, train_val_range_dict, train_target_Ser = getBeroreProcessDf(act_train_Df, None, True, None, train_target_Ser)
# デバッグ用
#act_train_Df_2, drop_columns, train_val_range_dict = getBeroreProcessDf(act_train_Df[:1000], drop_columns, True, train_val_range_dict)

In [ ]:
dispalyDf(act_train_Df_2, True, 10, "act_train_Df_2")

In [ ]:
train_target_Ser.value_counts()

In [ ]:
###### 特長量のカラム名のディクショナリを作る
col_name_dict = {
    index: col_name
    for index, col_name in  enumerate(act_train_Df_2.columns.tolist())
}

In [ ]:
act_test_Df_2, _, _, _ = getBeroreProcessDf(act_test_Df, drop_columns, False, train_val_range_dict, None)

In [ ]:
dispalyDf(act_test_Df_2, True, 10, "act_test_Df_2")

In [ ]:
####################################
##### 次元削減(特徴量選択) を使用する
##### ↓特徴量選択
####################################
def getSelectorFeature(inputFeatureDf,y_Df,feature_num, selector_step, model):
    
    return inputFeatureDf

# 以降の処理を首絞め
#     print("特徴量選択数 = ",feature_num)
#     if(feature_num < 0):
#         # デフォルト値
#         print("n_features_to_select = None に設定")
#         feature_num = None
#     elif feature_num > len(inputFeatureDf.columns.tolist()):
#         print ("feature_num({}) is Over colnum({})".format(feature_num, len(inputFeatureDf.columns.tolist())))
#         # 次元削減せずに返却する
#         return inputFeatureDf
    
#     # 特徴量選択
#     # step:特徴量削除の速度。一度の再帰処理により指定ステップ分の特徴量が消滅する。
# #     selector = RFE(estimator=LogisticRegression(penalty='l2', C=1.5, random_state=RANDOM_STATE_VAL, max_iter=120,
# #                                            solver='liblinear'),
#     selector = RFE(estimator=model,
#                    n_features_to_select=feature_num,
#                    step=selector_step)
#     selector.fit(inputFeatureDf,y_Df.values.ravel())
#     #selector.fit(inputFeatureDf, y_Df.iloc[:,0])

#     # 削除対象配列(Falseが削除対象)
#     selFlagArray = selector.support_
    
#     # 現状のカラム名を取得
#     columnIndex = inputFeatureDf.columns
    
#     # 削除対象カラム名リスト
#     drop_column_list = list()
#     # 削除対象配列 でループ(i=0,1,2,・・・)
#     #print(len(selFlagArray))
#     for i in range(len(selFlagArray)):
#         # Falseの場合に、削除対象リストに追加
#         if(selFlagArray[i] == False):
#             drop_column_list.append(columnIndex[i])

#     # 不要な列削除
#     #print("削除対象カラム = " , drop_column_list)
#     inputFeatureNewDf = inputFeatureDf.drop(drop_column_list, axis=1)
    

#     return inputFeatureNewDf


In [ ]:
# Pipelineに使う独自関数
class MyStandardScaler(object):
    def fit(self, X, y):
        print(" MyStandardScaler.fit")
        return X
    def fit_transform(self, X, y):
        print(" MyStandardScaler.fit_transform")
        return getStandardScalerDf(X)
#    def transform(self, X, y):
    def transform(self, X):
        print(" MyStandardScaler.transform")
        return getStandardScalerDf(X)


In [ ]:
# Pipelineに使う独自関数
class MyPreprocessor(object,):
    def fit(self, X, y):
        print(" MyPreprocessor.fit")
        return X
    def fit_transform(self, X, y, feature_num, selector_step, model):
        print(" MyPreprocessor.fit_transform")
        return getSelectorFeature(X,y,feature_num, selector_step, model)
#        return getSelectorFeature(X)
    def predict(self, X, train_col_list):
        print(" MyPreprocessor.predict")
        return getAdjustTestColDf(X, train_col_list)

In [ ]:
# 訓練データを学習用とバリデーション用に分ける
X_train_feature, X_train_validation, y_train_answer, y_train_validation = train_test_split(act_train_Df_2,     # 特徴量
                                             train_target_Ser,    # 正解データ
                                             test_size=TEST_SIZE_VAL ,
                                             random_state=RANDOM_STATE_VAL)

In [ ]:
dispalyDf(X_train_feature, True, 10, "X_train_feature")

In [ ]:
y_traind_df = pd.DataFrame(y_train_validation)
print(y_traind_df)
print(y_traind_df.dtypes)

print(y_traind_df[y_traind_df["target"]==1].index)

In [ ]:
print(y_train_answer.value_counts())
print(y_train_validation.value_counts())

In [ ]:
#### 学習 ####
model_def_dict = {
    ##### LGBMClassifier #####
#    "LGBMClassifier_Default":   LGBMClassifier(random_state=RANDOM_STATE_VAL),
    "LGBMClassifier_max_depth_5":   LGBMClassifier(random_state=RANDOM_STATE_VAL, max_depth=5),

# ボツ
#    "LGBMClassifier_max_depth_3":   LGBMClassifier(random_state=RANDOM_STATE_VAL, max_depth=3),
#    "LGBMClassifier_max_depth_4":   LGBMClassifier(random_state=RANDOM_STATE_VAL, max_depth=4),
#    "LGBMClassifier_max_depth_6":   LGBMClassifier(random_state=RANDOM_STATE_VAL, max_depth=6),
#    "LGBMClassifier_learning_rate_0.01":   LGBMClassifier(random_state=RANDOM_STATE_VAL, learning_rate=0.01),
    
    

    ##### XGBClassifier #####
#     "XGBClassifier_Default":   XGBClassifier(random_state=RANDOM_STATE_VAL),
#     "XGBClassifier_max_depth_3":   XGBClassifier(random_state=RANDOM_STATE_VAL, max_depth=3),

# ボツ
#    "XGBClassifier_max_depth_4":   XGBClassifier(random_state=RANDOM_STATE_VAL, max_depth=4),
#    "XGBClassifier_max_depth_5":   XGBClassifier(random_state=RANDOM_STATE_VAL, max_depth=5),

    
    ##### CatBoostClassifier #####
    # ハイパーパラメータの調整はしすぎるとダメ！！
    #early_stopping_roundsは小さすぎると学習不足につながる。むしろ指定しないほうがいい。
    
#    "CatBoostClassifier_Default": CatBoostClassifier(random_seed=RANDOM_STATE_VAL),
    
#     "CatBoostClassifier_200": CatBoostClassifier(random_seed=RANDOM_STATE_VAL,
#                                                  eval_metric="AUC",
#                                                 learning_rate=0.0156,
#                                                  early_stopping_rounds=300,
# #                                                 iterations = 20000),
#                                                  iterations = 2000),

                                                
##############    
#     ExtraTreesClassifier(n_estimators = ESTIMATOR_NUM),
#     RandomForestClassifier(n_estimators=ESTIMATOR_NUM, random_state=RANDOM_STATE_VAL),
#     DecisionTreeClassifier(),
#     LogisticRegression(penalty='l2', C=1.5, random_state=RANDOM_STATE_VAL,max_iter=120,
#                        solver="liblinear"),
#     LogisticRegression(penalty='l2', C=1.5, random_state=RANDOM_STATE_VAL,max_iter=120,
#                        solver="newton-cg"),
#     LogisticRegression(penalty='l2', C=1.5, random_state=RANDOM_STATE_VAL,max_iter=1200,
#                        solver="newton-cg"),
#     LogisticRegression(penalty='l2', C=1.0, random_state=RANDOM_STATE_VAL,max_iter=2000,
#                        solver="liblinear"),
}



model_fit_dict = {}

pipeline = Pipeline([
#     ('MyStandardScaler', MyStandardScaler()),   #標準化
     ('MyPreprocessor', MyPreprocessor()),   #独自関数
])

for model_name, model in model_def_dict.items():
    
    start = time.time()
    print("====================== {}  ======================".format(model_name))
    X_train_feature_new = pipeline.fit_transform(X_train_feature, y_train_answer,
                                     MyPreprocessor__feature_num = FEATURE_NUM,
                                     MyPreprocessor__selector_step = SELECTOR_STEP,
                                     MyPreprocessor__model = model,
                                    )
    x_col_name_list =  X_train_feature_new.columns.tolist()
    
    
    if "CatBoostClassifier" in model_name:
        ##### CatBoostClassifier #####
    #     model.fit(X_train_feature, y_train_answer.values.ravel(),
    # #              eval_set=(X_train_validation, y_train_validation.values.ravel()),
    #               eval_set=(getAdjustTestColDf(X_train_validation, X_train_feature.columns.tolist()),
    #                         y_train_validation.values.ravel()),
    #               cat_features=[2,3,4,5,6,7,8,21],
    #               plot=True
    #              )
        model.fit(X_train_feature_new, y_train_answer,
                  eval_set=(getAdjustTestColDf(X_train_validation, x_col_name_list),
                            y_train_validation),
                  cat_features=[x_col_name_list.index(col_name) for col_name in x_col_name_list
                                if col_name.endswith("_cat") or col_name.endswith("_bin")],
                  plot=True
                 )

        # ハイパーパラメータの表示
        print("BestScore = ", model.get_best_score())
        print(model.get_all_params())
        print("最適な木の数 = ",model.tree_count_)
        print(model.feature_importances_)
    elif "XGBClassifier" in model_name or "LGBMClassifier" in model_name:
        ##### XGBClassifier #####
        ##### LGBMClassifier #####
        model.fit(X_train_feature_new, y_train_answer)
        y_pred=model.predict_proba(getAdjustTestColDf(X_train_validation, x_col_name_list))
        print("auc_score = ", roc_auc_score(y_train_validation.values.ravel(), 
                                            y_pred[:,1]))

    ### model append ###
    model_fit_dict[model_name] =  model
print("----------------------------------------------------------------------------------------")

In [ ]:
total_preds = []

#### 予測 ####
for model_name, model in model_fit_dict.items():
    print("======================= {} =======================".format(model_name))
    
    
    if "CatBoostClassifier" in model_name:
        ##### CatBoostClassifier #####
        preds=model.predict(getAdjustTestColDf(act_test_Df_2, X_train_feature_new.columns.tolist()), prediction_type="Probability")
    elif "XGBClassifier" in model_name or "LGBMClassifier" in model_name:
        ##### XGBClassifier #####
        ##### LGBMClassifier #####
        preds=model.predict_proba(getAdjustTestColDf(act_test_Df_2, X_train_feature_new.columns.tolist()))
    
    
    
    if len(total_preds) > 0:
        total_preds = total_preds + preds
    else:
        total_preds = preds

#####
# モデル数で割る
mean_preds = total_preds / len(model_fit_dict)

In [ ]:
mean_preds

In [ ]:
target_Df = pd.DataFrame(
    {
#        target_col_name: [1 if row[1] >  PROBA_THRESHOLD else 0 for row in  mean_preds] 
        target_col_name: [row[1] for row in  mean_preds] 
    }
    ,dtype=int
)

In [ ]:
dispalyDf(target_Df, True, 10, "target_Df")

In [ ]:
target_Df.value_counts()

In [ ]:
output_Df = test_id_Df.join(target_Df)

In [ ]:
dispalyDf(output_Df, True, 10, "output_Df")

In [ ]:
# ディレクトリなければ作る
output_dir_path = os.path.join(os.getcwd(), 'output')
print(output_dir_path)
if not os.path.exists(output_dir_path):
    os.makedirs(output_dir_path) 

In [ ]:
# 出力
output_Df.to_csv('output/submission.csv',
                 sep=",",
                 header=True,
                 index=False,
                 mode="w",
                 encoding="utf-8",
                 line_terminator="\n"
)

In [ ]:
########################################################

In [ ]:
raise Exception("END♪")

In [ ]:
########################################################